In [123]:
ls ../../complete

12_graphs/ 16_graphs/ 20_graphs/ 24_graphs/ 4_graphs/  8_graphs/


In [124]:
import os
from classical_optimization.terra.utils import read_graph
path = '../../complete'
all_data = {}
qubit_numbers = []
for folder in os.listdir(path):
    num_qubits = int(folder.split('_')[0])
    qubit_numbers.append(num_qubits)
    for graph_file in os.listdir(os.path.join(path, folder)):
        graph_data = read_graph(os.path.join(path, folder, graph_file))
        extant_data = all_data.get(num_qubits, [])
        extant_data.append(graph_data)
        # Probably don't need this, since it's the same list...
        all_data[num_qubits] = extant_data

In [126]:
import matplotlib.pyplot as plt
for num_qubits in qubit_numbers:
    anneal_ratios = []
    es_ratios = []
    for graph_data in all_data[num_qubits]:
        if graph_data.get('annealing_result') is not None:
            annealing_approx_ratio = graph_data.get('annealing_result')[1]/graph_data.get('maxcut_result')[0]
            es_approx_ratio = graph_data.get('es_result')[1]/graph_data.get('maxcut_result')[0]
            anneal_ratios.append(annealing_approx_ratio)
            es_ratios.append(es_approx_ratio)
    if len(anneal_ratios) > 0:
        plt.hist(anneal_ratios, label='anneal')
        plt.hist(es_ratios, label='es')
        plt.xlabel('Approximation Ratio')
        plt.ylabel('Number of Graphs')
        plt.legend()
    plt.show()

# Time to figure out how to plot landscapes...

In [127]:
def plot_landscape(landscape, max_gamma, max_beta, min_gamma, min_beta, scatterpoints=None):
    ax = plt.imshow(landscape)
    if scatterpoints is None:
        scatterpoints = []
    for scatterpoint in scatterpoints:
        plt.scatter(scatterpoint[0][1], scatterpoint[0][0], c=scatterpoint[1], s=100, marker='*')
    ax.figure.canvas.draw()

    subplot = ax.figure.get_axes()[0]

    ticks = subplot.get_xticklabels()
    scale = 1/max([float(tick.get_text().replace('−', '-')) for tick in ticks if tick.get_text() != ''])
    scale *= max_gamma - min_gamma
    for tick in ticks:
        if tick.get_text() != '':
            tick.set_text(round(scale * float(tick.get_text().replace('−', '-')) - max_gamma, 2))
        subplot.set_xticklabels(ticks)

    ticks = subplot.get_yticklabels()
    scale = 1 / max([float(tick.get_text().replace('−', '-')) for tick in ticks if tick.get_text() != ''])
    scale *= max_beta - min_beta
    for tick in ticks:
        if tick.get_text() != '':
            tick.set_text(round(scale * float(tick.get_text().replace('−', '-')) - max_beta, 2))
        subplot.set_yticklabels(ticks)

    plt.ylabel(r"$\beta$")
    plt.xlabel(r"$\gamma$")

In [128]:
all_data[4]

[{'graph': <networkx.classes.graph.Graph at 0x12e07a490>,
  'landscape_d40_b0.7853981633974483_g3.141592653589793_b-0.7853981633974483_g-3.141592653589793': array([[ 0.00000000e+00,  1.35489632e+00,  4.65880836e+00, ...,
          -1.57937165e+00, -4.53194887e-01, -4.44089210e-16],
         [ 0.00000000e+00,  4.27182251e-01,  3.03511983e+00, ...,
          -1.75686158e+00, -6.00909392e-01, -2.88657986e-15],
         [ 4.71844785e-16, -4.94050067e-01,  1.39315365e+00, ...,
          -1.90930217e+00, -7.38929414e-01, -4.44089210e-15],
         ...,
         [ 0.00000000e+00,  3.13411493e+00,  7.68470421e+00, ...,
          -1.16816104e+00, -1.44137339e-01,  1.77635684e-15],
         [ 0.00000000e+00,  2.26506485e+00,  6.22216653e+00, ...,
          -1.38142927e+00, -2.99611628e-01,  1.11022302e-15],
         [ 0.00000000e+00,  1.35489632e+00,  4.65880836e+00, ...,
          -1.57937165e+00, -4.53194887e-01, -1.33226763e-15]]),
  'landscape_d40_b0.7853981633974483_g3.141592653589793_b-0.7

In [120]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
for num_qubits in qubit_numbers:
    for graph_data in all_data[num_qubits]:
        landscape = None
        for k in graph_data.keys():
            if 'landscape' in k and 'time' not in k:
                _, disc, max_beta, max_gamma, min_beta, min_gamma = k.split('_')
                disc = int(disc[1:])
                max_beta = float(max_beta[1:])
                min_beta = float(min_beta[1:])
                max_gamma = float(max_gamma[1:])
                min_gamma = float(min_gamma[1:])
                landscape = graph_data[k]
        if landscape is not None:
            ind = np.argmax(landscape)
            ind = np.unravel_index(ind,landscape.shape)
            scatterpoints = [(ind, 'r')]
            if graph_data.get('annealing_result') is not None:
                gamma, beta = graph_data.get('annealing_result')[0]
                y, x = (beta % (max_beta-min_beta) * (disc - 1) / max_beta) + min_beta, (gamma % (max_gamma - min_gamma) * (2*disc-1)/max_gamma) + min_gamma
                scatterpoints.append(((y, x), 'w'))
            plot_landscape(landscape, max_gamma, max_beta, min_gamma, min_beta, scatterpoints)  
            plt.show()

In [122]:
all_data[4]

[{'graph': <networkx.classes.graph.Graph at 0x127422110>,
  'annealing_result': (array([-2.46913912, -0.70484767]), 16.7),
  'es_result': (array([ 0.0028566, -0.0062139]), 20.1),
  'maxcut_result': (21, [0, 2], [1, 3]),
  'maxarg_result': (2564, 16.710741055070585)},
 {'graph': <networkx.classes.graph.Graph at 0x111494e50>,
  'annealing_result': (array([-2.86285753,  2.65209056]), 196.3),
  'es_result': (array([-0.0089756 ,  0.00676812]), 236.9),
  'maxcut_result': (251, [1, 2], [0, 3]),
  'maxarg_result': (515, 168.83493893405497)},
 {'graph': <networkx.classes.graph.Graph at 0x10f1a0650>,
  'annealing_result': (array([-2.16856977, -0.25199734]), 87.8),
  'es_result': (array([-0.01852693,  0.01813491]), 118.4),
  'maxcut_result': (126, [2, 3], [0, 1]),
  'maxarg_result': (2917, 76.43128076451788)},
 {'graph': <networkx.classes.graph.Graph at 0x1114979d0>,
  'annealing_result': (array([4.40097623, 2.65209056]), 221.8),
  'es_result': (array([ 0.02079191, -0.00264236]), 260.5),
  'maxcu